Download data

In [1]:
# from google.colab import drive

# drive.mount('/content/gdrive')

In [2]:
# !ls gdrive/MyDrive/PED

In [ ]:
import pandas
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import nltk
nltk.download('punkt')
%matplotlib inline

df = pandas.read_csv('../../reddit_wsb.csv')
df["body"] = df["body"].fillna('')
df["title"] = df["title"].fillna('')

df

In [ ]:
import pickle
img_text = pickle.load(open("../../pickle/texts.pkl", "rb"))
print(img_text.keys())

In [ ]:
seconds_in_day = 60 * 60 * 24.
df['sin_time'] = np.sin(2*np.pi*(df.created % seconds_in_day)/seconds_in_day)
df['cos_time'] = np.cos(2*np.pi*(df.created % seconds_in_day)/seconds_in_day)

Data summary

In [ ]:
df.describe()

In [ ]:
df.sample(5000).plot.scatter('sin_time','cos_time').set_aspect('equal')

In [ ]:
df.columns

In [ ]:
plt.hist(df['score'], bins=30)
plt.ylabel('Count')
plt.yscale('log')
plt.xlabel('Data');

In [ ]:
plt.hist(df['created'], bins=30)
plt.ylabel('Count')
plt.yscale('log')
plt.xlabel('Data')

Text processing

In [ ]:
from collections import Counter
import string
import emoji
import re
import spacy
from spacy.lang.en.examples import sentences 

nlp = spacy.load("en_core_web_sm")

Most common words

In [ ]:
try:
  df['body_lemmatized'] = pandas.read_pickle('../../pickle/body_lemmatized.pkl')
except:
  pass
try:
  df['title_lemmatized'] = pandas.read_pickle('../../pickle/title_lemmatized.pkl')
except:
  pass

In [ ]:
# x = None
# try: 
#   x = pandas.read_pickle('gdrive/MyDrive/PED/title_lemmatized2.pkl')
# except:
#   pass
'title_lemmatized' in df

In [ ]:
df['body_lemmatized'].head(5)

In [ ]:
if 'body_lemmatized' not in df: df['body_lemmatized'] = df['body'].apply(lambda x: [i.lemma_ for i in nlp(x.lower()) if (not i.is_stop) and len(i.lemma_) > 0])
if 'title_lemmatized' not in df: df['title_lemmatized'] = df['title'].apply(lambda x: [i.lemma_ for i in nlp(x.lower()) if (not i.is_stop) and len(i.lemma_) > 0])
# df['title_lemmatized'].to_pickle("gdrive/MyDrive/PED/title_lemmatized.pkl")

In [ ]:
' '.join([' '.join(x) for x in img_text.values()])

In [ ]:
len(img_text.keys())

In [ ]:
text_lens_no_0 = [len(x) for x in img_text.values() if len(x) > 0]
print('count', len(img_text))
print('no text in', len([len(x) for x in img_text.values() if len(x) == 0]), 'images')
print('min', np.min([len(x) for x in img_text.values()]))
print('max', np.max([len(x) for x in img_text.values()]))

print('q1', np.quantile(text_lens_no_0, 0.25))
print('median', np.median(text_lens_no_0))
print('q3', np.quantile(text_lens_no_0, 0.75))


In [ ]:
words_separate = [' '.join(x).split(' ') for x in img_text.values()]

text_lens_no_0 = [len(x) for x in img_text.values() if len(x) > 0]
print('min', np.min([len(x) for x in words_separate]))
print('max', np.max([len(x) for x in words_separate]))
print('q1', np.quantile([len(x) for x in words_separate], 0.25))
print('median', np.median([len(x) for x in words_separate]))
print('q3', np.quantile([len(x) for x in words_separate], 0.75))


In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
img_most_common = Counter(word_tokenize(' '.join([' '.join(x) for x in img_text.values()])))
for key, cnts in list(img_most_common.items()):
  if not key in emoji.UNICODE_EMOJI and (key in stopwords.words('english') or not re.match(r'\w+',key)):
    del img_most_common[key]
img_most_common = img_most_common.most_common(100)
img_most_common


In [ ]:
import pickle

In [ ]:
body_most_common = None
try:
  body_most_common = pickle.load(open('../../pickle/body_most_common.pkl', "rb"))
except:
  pass

if body_most_common is None: body_most_common = Counter(df["body_lemmatized"].sum())



In [ ]:
def prune(most_common_words):
  for key, cnts in list(most_common_words.items()):
    if not key in emoji.UNICODE_EMOJI and (not re.match(r'\w+',key) or len(key) < 2):
      del most_common_words[key]
  for i in ['don\\u2019', 'i\\u2019', 'it\\u2019s']:
    del most_common_words[i]
# body_most_common = body_most_common.most_common(100)
prune(body_most_common)
body_most_common.most_common(100)

In [ ]:
title_most_common = None
try:
  title_most_common = pickle.load(open('../../pickle/title_most_common.pkl', "rb"))
except:
  pass

if title_most_common is None: title_most_common = Counter(df["title_lemmatized"].sum())

# title_most_common = Counter(df["title_lemmatized"].sum())
# for key, cnts in list(title_most_common.items()):
#   if not key in emoji.UNICODE_EMOJI and (key in stopwords.words('english') or not re.match(r'\w+',key)):
#     del title_most_common[key]
# title_most_common = title_most_common.most_common(100)
prune(title_most_common)
title_most_common.most_common(100)

Parameters creation

In [ ]:
additional_keywords = ['gme', 'hold', 'buy', 'retard', 'wife', 'hedgies', '🚀', 'moon','🧻','🙏', '💎', '🤲','✋','🙌🏼', '🦍', '👋🏼']
count = lambda l1,l2: sum([1 for x in l1 if x in l2])
text_attributes = {}
for key in additional_keywords: 
  text_attributes['title_' + key] =  df["title_lemmatized"].apply(lambda x: x.count(key))
  text_attributes['body_' + key] = df["body_lemmatized"].apply(lambda x: x.count(key))




In [ ]:
 df["title_lemmatized"].head(5).apply(lambda x: x.count('retard'))

In [ ]:
for key, cnts in body_most_common.most_common(100): 
  text_attributes['body_' + key.lower()] = df["body_lemmatized"].apply(lambda x: x.count(key))


for key, cnts in title_most_common.most_common(100): 
  text_attributes['title_' + key.lower()] = df["title_lemmatized"].apply(lambda x: x.count(key))


In [ ]:

text_attributes['body_punctuation'] = df['body'].apply(lambda s: count(s, string.punctuation))
text_attributes['title_punctuation'] = df['title'].apply(lambda s: count(s, string.punctuation))

text_attributes['body_cap_ratio'] = df['body'].str.count(r'[A-Z]')/df['body'].str.count(r'[a-zA-Z]')
text_attributes['title_cap_ratio'] = df['title'].str.count(r'[A-Z]')/df['title'].str.count(r'[a-zA-Z]')

text_attributes['time_of_day'] = pandas.to_datetime(df['timestamp']).dt.hour + pandas.to_datetime(df['timestamp']).dt.minute/60

text_attributes['title_length'] = df['title'].apply(len)
text_attributes['title_non_alphanumeric_ratio'] = df['title'].str.count(r'[^A-Za-z0-9]')/df['title'].apply(len)


In [ ]:
def regex_count_in_url_and_body(regex):
    return df['body'].apply(lambda s: len(re.findall(regex, s))) + df['url'].apply(lambda s: len(re.findall(regex, s)))

In [ ]:
text_attributes['link'] = regex_count_in_url_and_body(r'(https?://[^\s]+)')
text_attributes['reddit_link'] = regex_count_in_url_and_body(r'(https?:\/\/w?w?w?\.?redd\.?it[^\s]+)')
text_attributes['yt_link'] = regex_count_in_url_and_body(r'(https?:\/\/w?w?w?\.?youtu\.?be[^\s]+)')
text_attributes['tweet_link'] = regex_count_in_url_and_body(r'(https?:\/\/w?w?w?\.?twitter[^\s]+)')
text_attributes['facebook_link'] = regex_count_in_url_and_body(r'(https?:\/\/w?w?w?\.?facebook[^\s]+)')

text_attributes['gif'] = regex_count_in_url_and_body(r'(https?:\/\/w?w?w?\.?giphy[^\s]+)') + regex_count_in_url_and_body(r'(https?:\/\/w?w?w?\.?gifyu[^\s]+)') + regex_count_in_url_and_body(r'(https?:\/\/w?w?w?\.?gfycat[^\s]+)') + df['url'].apply(lambda s: len(re.findall(r'(\.gif)', s)))

text_attributes = pandas.DataFrame(text_attributes)
text_attributes
df[text_attributes['gif']>=1]

In [ ]:
text_attributes.describe()

In [ ]:
df.describe()

In [ ]:
for i in ['score', 'sin_time', 'cos_time']:
  text_attributes[i] = df[i]  
text_attributes['day'] = pandas.to_datetime(df['timestamp']).dt.day
text_attributes['month'] = pandas.to_datetime(df['timestamp']).dt.month
text_attributes.describe()

In [ ]:
print(pandas.to_datetime(df['timestamp'].head(10)))
print(pandas.to_datetime(df['timestamp'].head(10)).dt.day)
print(pandas.to_datetime(df['timestamp'].head(10)).dt.month)

In [ ]:
print(re.findall('http','ahttpjkshdoifjdshttp'))
print()
print(re.findall(r"https?://(?:(?:[^\s()])|(?:\(\S*\)]*\)))+", 'https://www.benzinga.com/government/21/01/19337399/something-systemically-wrong-with-gamestop-options-trading-says-massachusetts-securities-regulator(dupa=dupa()))'))

In [ ]:
text_attributes['body_🙏'].describe()
text_attributes['score'].min() == text_attributes['score'].max() 

In [ ]:
to_be_removed = []
for column in text_attributes:
  if text_attributes[column].min() == text_attributes[column].max():
    to_be_removed.append(column)
len(to_be_removed) # 11

In [ ]:
for i in to_be_removed:
  del text_attributes[i]

In [ ]:
text_attributes['day'].hist()


In [ ]:
text_attributes['day'] = text_attributes['day'].apply(lambda x: x+(31-19) if x < 19 else x-19)

In [ ]:
del text_attributes['month']

In [ ]:
del text_attributes['time_of_day']

In [ ]:
# for column in text_attributes:
#     print(column, text_attributes[column].corr(df['score']))

Korelacja

In [ ]:
corrMatrix = text_attributes.corr(method='spearman')

In [ ]:
corrMatrixSave = corrMatrix.copy()

In [ ]:
x = corrMatrix['score'][corrMatrix['score']<1]
print(x.min(), x.keys()[x.argmin()])
print(x.max(), x.keys()[x.argmax()])
# with pandas.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#   print(x.sort_values())


In [ ]:
treshold = .5
for i,column in enumerate(corrMatrix):
  x = corrMatrix[column][i+1:]
  if x.min() < -treshold:
    print(x.min(), column, x.keys()[x.argmin()])
  if x.max() > treshold:
    print(x.max(), column, x.keys()[x.argmax()])

In [ ]:
treshold = .5
correlations = []
for i,column in enumerate(corrMatrix):
  x = corrMatrix[column][i+1:]
  for row,v in x.items():
    correlations.append([v,column,row])
correlations.sort(key = lambda x: -abs(x[0]))

In [ ]:
len(correlations)

In [ ]:
correlations.sort(key = lambda x: -abs(x[0]))

In [ ]:
len(text_attributes.columns)

In [ ]:
text_attributes2 = text_attributes.copy()

In [ ]:
text_attributes = text_attributes2.copy()

In [ ]:
score_corr = corrMatrix['score'][corrMatrix['score']<1]
for corr in correlations:
  if len(text_attributes.columns) <= 30: break
  c,a1,a2 = corr
  if 'score' in [a1,a2]: continue
  if a1 in text_attributes and a2 in text_attributes:
    to_be_removed = a1 if score_corr[a1] < score_corr[a2] else a2
    del text_attributes[to_be_removed]

In [ ]:
text_attributes

In [ ]:
text_attributes.columns

In [ ]:
score_corr_text = corrMatrix['score']
lista1=[]
for i in text_attributes.columns:
  lista1.append([score_corr_text[i], i])
  # print('{:10.5f}'.format(score_corr[i]), class_dict[int(i)])
lista1.sort(key=lambda x: -abs(x[0]))
for i in lista1:
  print('{:10.5f}'.format(i[0]), i[1])

In [ ]:
corrMatrix

Obróbka atrybutów ze zdjęć

In [ ]:
img_class = pickle.load(open("../../pickle/image_classes.pkl", "rb"))
img_clusters = pickle.load(open("../../pickle/image_color_clusters.pkl", "rb"))
img_means = pickle.load(open("../../pickle/image_hsv_means.pkl", "rb"))

pic_data = {}
pic_data['id'] = []
pic_data['text'] = []
pic_data['probabilities'] = []
pic_data['clusters'] = []
pic_data['means'] = []

for key in img_class.keys():
  pic_data['id'].append(key)
  pic_data['text'].append(" ".join(img_text[key[:-4]]))
  pic_data['probabilities'].append(img_class[key])
  pic_data['clusters'].append(img_clusters[key])
  pic_data['means'].append(img_means[key])

pic_data = pandas.DataFrame(pic_data)
pic_data

pic_attributes = {'id': []}
for key, cnts in img_most_common: 
  pic_attributes['id'].append(key)
  pic_attributes['text_' + key] =  pic_data['text'].str.lower().str.count(key.lower())


In [ ]:
img_clusters = pickle.load(open("../../pickle/image_color_clusters.pkl", "rb"))
img_means = pickle.load(open("../../pickle/image_hsv_means.pkl", "rb"))

pic_data = {}
pic_data['id'] = []
pic_data['text'] = []
pic_data['probabilities'] = []
pic_data['clusters'] = []
pic_data['means'] = []

for key in img_means.keys():
  pic_data['id'].append(key)
  pic_data['text'].append(" ".join(img_text[key[:-4]]))
  pic_data['probabilities'].append(img_class[key])
  pic_data['clusters'].append(img_clusters[key])
  pic_data['means'].append(img_means[key])

pic_data = pandas.DataFrame(pic_data)

pic_attributes = {'id': pic_data['id']}
for key, cnts in img_most_common: 
  pic_attributes['text_' + key] =  pic_data['text'].str.lower().str.count(key.lower())

pic_attributes['h_sin'] = pic_data['means'].apply(lambda s:  np.sin(2*np.pi*(s[0]/255)))
pic_attributes['h_cos'] = pic_data['means'].apply(lambda s:  np.sin(2*np.pi*(s[0]/255)))
pic_attributes['s'] = pic_data['means'].apply(lambda s: s[1]/255)
pic_attributes['v'] = pic_data['means'].apply(lambda s: s[2]/255)

for j in range(5):
  for i in range(3):
    pic_attributes['color_'+str(j)+'_'+'rgb'[i]] = pic_data['clusters'].apply(lambda s: s['color'][j][i])
  pic_attributes['color_'+str(j)+'_%'] = pic_data['clusters'].apply(lambda s: s['percentage'][j])

pic_attributes = pandas.DataFrame(pic_attributes)
print(pic_attributes)